In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils.utils import *

px.set_mapbox_access_token("pk.eyJ1IjoiamdhY29zdGFzIiwiYSI6ImNsYWJwd3g1ZDAwaGUzb3Q0ZG04NDNndGgifQ.brk6kVA6biVSH0ovZ1dreA")

In [ ]:
months = os.listdir('../data/comed_month')

In [ ]:
df19 = pd.DataFrame()
df20 = pd.DataFrame()
df21 = pd.DataFrame()
for month in months[8:]:
    df_data = pd.read_csv(f'../data/comed_month/{month}', parse_dates=['date_time'])
    df_data = df_data[df_data.service_name.str.contains('RESIDENTIAL')]
    df_data['day'] = df_data.date_time.dt.date
    df = df_data.groupby(['zip5', 'day'])[['energy']].sum().reset_index()
    df_acc = df_data.groupby(['zip5', 'day', 'service_name'])['n_acc'].mean().reset_index().groupby(['zip5', 'day']).n_acc.sum().reset_index()
    df = pd.merge(df, df_acc, on=['zip5', 'day'])
    df = df.groupby('day')[['energy', 'n_acc']].sum().reset_index()
    if '2019' in month:
        df19 = pd.concat([df19, df]) 
    if '2020' in month:
        df20 = pd.concat([df20, df])
    if '2021' in month:
        df21 = pd.concat([df21, df])

    
    

In [ ]:
df_plot = pd.merge(df19.assign(grouper=pd.to_datetime(df19['day']).dt.dayofyear),
               df20.assign(grouper=pd.to_datetime(df20['day']).dt.dayofyear),
               how='left', on='grouper', suffixes=('_2019', '_2020'))
df_plot = pd.merge(df_plot,
               df21.assign(grouper=pd.to_datetime(df21['day']).dt.dayofyear),
               how='left', on='grouper', suffixes=('', '_2021'))

df_plot = df_plot.rename(columns={'n_acc': 'n_acc_2021', 'energy': 'energy_2021'})


In [ ]:
colors = px.colors.qualitative.Safe
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2019, name="2019 electricity usage", marker_color=colors[0]),secondary_y=False,)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2020, name="2020 electricity usage", marker_color=colors[1]),secondary_y=False,)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2021, name="2021 electricity usage", marker_color=colors[2]),secondary_y=False,)

fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2020, name="2020 ComEd accounts", marker_color=colors[4]), secondary_y=True)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2019, name="2019 ComEd accounts", marker_color=colors[3]), secondary_y=True)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2021, name="2021 ComEd accounts", marker_color=colors[5]), secondary_y=True)

fig.update_xaxes(dtick="M1",tickformat="%b", ticklabelmode="period")

fig.update_layout(title_text="2019- 2020 Non-Residential Daily Electricity Usage")

# Set x-axis title
fig.update_xaxes(title_text="Month")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity usage (kWh)", secondary_y=False)
fig.update_yaxes(title_text="Number of accounts", secondary_y=True)


fig.show()

In [ ]:
colors = px.colors.qualitative.Plotly
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2019, name="2019 electricity usage", marker_color=colors[0]),secondary_y=False,)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2020, name="2020 electricity usage", marker_color=colors[1]),secondary_y=False,)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.energy_2021, name="2021 electricity usage", marker_color=colors[2]),secondary_y=False,)

fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2020, name="2020 ComEd accounts", marker_color=colors[4]), secondary_y=True)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2019, name="2019 ComEd accounts", marker_color=colors[3]), secondary_y=True)
fig.add_trace(go.Scatter(x=df_plot.day_2019, y=df_plot.n_acc_2021, name="2021 ComEd accounts", marker_color=colors[5]), secondary_y=True)

fig.update_xaxes(dtick="M1",tickformat="%b", ticklabelmode="period")

fig.update_layout(title_text="2019- 2020 Non-Residential Daily Electricity Usage")

# Set x-axis title
fig.update_xaxes(title_text="Month")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity usage (kWh)", secondary_y=False)
fig.update_yaxes(title_text="Number of accounts", secondary_y=True)


fig.show()